# Day 10 - Decorators and Context Managers in Python

## Why Are Decorators and Context Managers Important?

Decorators are powerful tools in Python that allow you to enhance the functionality of existing functions or methods in a clean, elegant, and reusable manner. They are particularly useful for adding common "wrapper" functionalities, such as performance monitoring, access control, or logging operations, without altering the original function body. This can greatly simplify your code and make it easier to maintain.

Context managers, on the other hand, play a crucial role in resource management. They provide a structured way to allocate and release resources precisely when you need to. Whether it's dealing with file operations, network connections, or ensuring that locks are properly handled, context managers help prevent resource leaks and ensure that your application runs more reliably. By encapsulating the setup and teardown logic for resources, context managers make your code both safer and cleaner.


## Basic Theory: Understanding Decorators and Context Managers

### Decorators:

Decorators enhance the functionality of existing functions or methods in a clean and reusable way. Decorators are often used for logging, access control, memoization, and more. They are implemented using the @decorator_name syntax above a function definition.

Here's a basic syntax example:

```python
def my_decorator(func):
    def wrapper():
        print("Something is happening before the function is called.")
        func()
        print("Something is happening after the function is called.")
    return wrapper

@my_decorator
def say_hello():
    print("Hello!")

say_hello()
```

Let's break down the last code snippet:
- `my_decorator` is a function that takes another function as its argument.
- Inside `my_decorator`, the `wrapper` function is defined. It wraps the execution of the input function (`func`) with additional code before and after its call.
- The `@my_decorator` syntax is syntactic sugar that applies the decorator to the `say_hello` function.


### Context Managers:

A context manager is an object that defines the runtime context to be established when executing a `with` statement.

Context managers are used to manage resources such as file streams or network connections.

They ensure that resources are properly cleaned up after use, preventing resource leaks.

Here's how you can define one using a class:

```python
class ManagedFile:
    def __init__(self, filename):
        self.filename = filename

    def __enter__(self):
        self.file = open(self.filename, 'w')
        return self.file

    def __exit__(self, exc_type, exc_val, exc_tb):
        if self.file:
            self.file.close()

with ManagedFile('hello.txt') as f:
    f.write('Hello, world!')
```

Let's now break down the above code:
- The `ManagedFile` class defines how to open and close a file. It uses the special methods `__enter__` and `__exit__` to implement the context management protocol.
- The `__enter__` method is called when the execution enters the context of the `with` statement and it opens the file and returns it.
- The `__exit__` method ensures that the file is closed when the block inside the `with` statement is exited, even if an exception occurs.


In [ ]:
# Example: Implementing a Simple Logging Decorator
import time

def log_execution_time(func):
    """Decorator that logs the execution time of a function."""
    def wrapper(*args, **kwargs):
        start_time = time.time()
        result = func(*args, **kwargs)
        end_time = time.time()
        execution_time = end_time - start_time
        print(f"Executed {func.__name__} in {execution_time:.4f} seconds")
        return result
    return wrapper

@log_execution_time
def slow_function(seconds):
    """Simulates a slow function by sleeping for a given number of seconds."""
    time.sleep(seconds)
    return f"Slept for {seconds} seconds"

print(slow_function(2))


In [ ]:
# Example: Managing Resource Usage with Context Managers
class ManagedFile:
    def __init__(self, filename):
        self.filename = filename

    def __enter__(self):
        self.file = open(self.filename, 'w')
        return self.file

    def __exit__(self, exc_type, exc_value, exc_traceback):
        if self.file:
            self.file.close()

# Using the context manager
with ManagedFile('example.txt') as f:
    f.write('Hello, world!')
